RNA-Seq workflow: gene-level exploratory analysis and differential expression
=============================================================================

Michael Love \[1\], Simon Anders \[2\], Wolfgang Huber \[2\]

\[1\] Department of Biostatistics, Dana-Farber Cancer Institute and Harvard School of Public Health, Boston, US;

\[2\] European Molecular Biology Laboratory (EMBL), Heidelberg, Germany.

Contents
--------

-   [Counting reads](#count)
-   [Building a DESeqDataSet](#construct)
-   [Exploratory analysis and visualization](#eda)
-   [Differential expression](#de)
-   [Plotting results](#plots)
-   [Annotation](#annotate)
-   [Accounting for unknown batches](#batch)
-   [Time series experiments](#time)

Introduction
------------

This lab will walk you through an end-to-end gene-level RNA-Seq differential expression workflow, using *[DESeq2](http://bioconductor.org/packages/release/bioc/html/DESeq2.html)* (Love, Huber, and Anders 2014) along with other Bioconductor packages. We will start from the FASTQ files, show how these were aligned to the reference genome, prepare a count matrix which tallies the number of RNA-seq reads/fragments within each gene for each sample, perform exploratory data analysis (EDA), perform differential gene expression analysis, and visually explore the results.

We note that a number of other Bioconductor packages can also be used for statistical inference of differential expression for RNA-seq at the gene level including *[edgeR](http://bioconductor.org/packages/release/bioc/html/edgeR.html)* (Robinson, McCarthy, and Smyth 2009), *[limma](http://bioconductor.org/packages/release/bioc/html/limma.html)* with the voom method (Law et al. 2014), *[DSS](http://bioconductor.org/packages/release/bioc/html/DSS.html)* (H. Wu, Wang, and Wu 2013), *[EBSeq](http://bioconductor.org/packages/release/bioc/html/EBSeq.html)* (Leng et al. 2013), and *[BaySeq](http://bioconductor.org/packages/release/bioc/html/BaySeq.html)* (Hardcastle and Kelly 2010).

If you have questions about this workflow, please post them to the [Bioconductor support site](https://support.bioconductor.org/). If the questions concern a specific package, you can tag the post with the name of the package, or for general questions about the workflow, tag the post with `deseq2`. Note the [posting guide](http://www.bioconductor.org/help/support/posting-guide/) for crafting an optimal question for the support site.

### Experimental data

The data used in this workflow is stored in the *[airway](http://bioconductor.org/packages/release/data/experiment/html/airway.html)* package, which summarizes an RNA-Seq experiment wherein airway smooth muscle cells were treated with dexamethasone, a synthetic glucocorticoid steroid with anti-inflammatory effects (Himes et al. 2014). Glucocorticoids are used, for example, in asthma patients to reduce inflammation of the airways. In the experiment, four primary human airway smooth muscle cell lines were treated with 1 micromolar dexamethasone for 18 hours. For each of the four cell lines, we have a treated and an untreated sample. Links to more descriptions of the experiment and raw data are provided below:

-   PMID: [24926665](http://www.ncbi.nlm.nih.gov/pubmed/24926665).
-   GEO: [GSE52778](http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE52778).

<a id="count"></a>

Preparing count matrices
------------------------

As input, the *DESeq2* statistical method expects count data as obtained, e.g., from RNA-Seq or another high-throughput sequencing experiment, in the form of a matrix of integer values. The value in the *i*-th row and the *j*-th column of the matrix tells how many reads (or fragments, for paired-end RNA-seq) can be uniquely assigend to gene *i* in sample *j*. Analogously, for other types of assays, the rows of the matrix might correspond e.g., to binding regions (with ChIP-Seq), species of bacteria (with metagenomic datasets), or peptide sequences (with quantitative mass spectrometry).

The values in the matrix must be raw counts of sequencing reads/fragments. This is important for *DESeq2*'s statistical model to hold, as only the raw counts allow assessing the measurement precision correctly. It is important to not provide counts which were pre-normalized for sequencing depth or library size, as the statistical model is most powerful when applied to raw counts, and is designed to account for library size differences internally.

### Aligning reads to a reference genome

The computational analysis of an RNA-Seq experiment begins earlier: we first obtain a set of FASTQ files that contain the nucleotide sequence of each read and a quality score at each position. These reads must first be aligned to a reference genome or transcriptome. It is important to know if the sequencing experiment was single-end or paired-end, as the alignment software will require the user to specify both FASTQ files for a paired-end experiment. The output of this alignment step is commonly stored in a file format called [SAM/BAM](http://samtools.github.io/hts-specs).

A number of software programs exist to align reads to a reference genome, and the development is too rapid for this document to provide an up-to-date list. We recommend consulting benchmarking papers that discuss the advantages and disadvantages of each software, which include accuracy, sensitivity in aligning reads over splice junctions, speed, memory footprint, and many other features.

The reads for this experiment were aligned to the Ensembl release 75 (Flicek et al. 2014) human reference genome using the [STAR read aligner](https://code.google.com/p/rna-star/) (Dobin et al. 2013):

[SAMtools](http://samtools.sourceforge.net/) (H. Li et al. 2009) was used to generate BAM files.

The BAM files for a number of sequencing runs can then be used to generate count matrices, as described in the following section.

### Locating alignment files

Besides the count matrix, which we will use later, the *[airway](http://bioconductor.org/packages/release/data/experiment/html/airway.html)* package also contains eight files with a small subset of the total number of reads in the experiment. The reads were selected which aligned to a small region of chromosome 1. We will use these files to demonstrate how a count matrix can be constructed from BAM files. Afterwards, we will load the full count matrix corresponding to all samples and all data, which is already provided in the same package, and will continue the analysis with that full matrix.

We load the data package with the example data:

In [ ]:
library("airway")

The R function *system.file* can be used to find out where on your computer the files from a package have been installed. Here we ask for the full path to the `extdata` directory, where R packages store external data, which is part of the *[airway](http://bioconductor.org/packages/release/data/experiment/html/airway.html)* package.

In [ ]:
dir <- system.file("extdata", package="airway", mustWork=TRUE)

In this directory, we find the eight BAM files (and some other files):

In [ ]:
list.files(dir)

Typically, we have a table with detailed information for each of our samples, and which links samples to the associated FASTQ and BAM files. For your own project, you might create such a comma-separated value (CSV) file using a text editor or spreadsheet software such as Excel.

We load such a CSV file with *read.csv*:

In [ ]:
csvfile <- file.path(dir,"sample_table.csv")
(sampleTable <- read.csv(csvfile,row.names=1))

**Note:** here and elsewhere in the workflow, the parentheses `()` around the entire code of the last line above is an R trick to print the output of the function in addition to saving it to `sampleTable`. This is equivalent to assigning and then showing the object in two steps:

In [ ]:
sampleTable <- read.csv(csvfile,row.names=1)
sampleTable

Once the reads have been aligned, there are a number of tools which can be used to count the number of reads/fragments which can be uniquely assigned to genomic features for each sample. These often take as input SAM/BAM alignment files and a file specifying the genomic features, e.g. a GFF3 or GTF file specifying the gene models.

The following tools can be used generate count matrices, *summarizeOverlaps* (Lawrence et al. 2013), *featureCounts* (Liao, Smyth, and Shi 2014), or *htseq-count* (Anders, Pyl, and Huber 2014):

| function            | package                                                                                          | framework      | output                 | *DESeq2* input function  |
|---------------------|--------------------------------------------------------------------------------------------------|----------------|------------------------|--------------------------|
| *summarizeOverlaps* | *[GenomicAlignments](http://bioconductor.org/packages/release/bioc/html/GenomicAlignments.html)* | R/Bioconductor | *SummarizedExperiment* | *DESeqDataSet*           |
| *featureCounts*     | *[Rsubread](http://bioconductor.org/packages/release/bioc/html/Rsubread.html)*                   | R/Bioconductor | matrix                 | *DESeqDataSetFromMatrix* |
| *htseq-count*       | [HTSeq](http://www-huber.embl.de/users/anders/HTSeq)                                             | Python         | files                  | *DESeqDataSetFromHTSeq*  |

Using the `Run` column in the sample table, we construct the full paths to the files we want to perform the counting operation on:

In [ ]:
filenames <- file.path(dir, paste0(sampleTable$Run, "_subset.bam"))
file.exists(filenames)

We indicate in Bioconductor that these files are BAM files using the *BamFileList* function from the *[Rsamtools](http://bioconductor.org/packages/release/bioc/html/Rsamtools.html)* package, which provides an R interface to BAM files. Here we also specify details about how the BAM files should be treated, e.g., only process 2 million reads at a time. See `?BamFileList` for more information.

In [ ]:
library("Rsamtools")
bamfiles <- BamFileList(filenames, yieldSize=2000000)

**Note:** make sure that the chromosome names of the genomic features in the annotation you use are consistent with the chromosome names of the reference used for read alignment. Otherwise, the scripts might fail to count any reads to features due to the mismatching names. We can check the chromosome names (here called "seqnames") in the alignment files like so:

In [ ]:
seqinfo(bamfiles[1])

### Defining gene models

Next, we need to read in the gene model which will be used for counting reads/fragments. We will read the gene model from an Ensembl [GTF file](http://www.ensembl.org/info/website/upload/gff.html) (Flicek et al. 2014), using *makeTxDbFromGFF* from the *[GenomicFeatures](http://bioconductor.org/packages/release/bioc/html/GenomicFeatures.html)* package. GTF files can be downloaded from Ensembl's FTP site or other gene model repositories. A *TxDb* object is a database which can be used to generate a variety of range-based objects, such as exons, transcripts, and genes. We want to make a list of exons grouped by gene for counting read/fragments.

There are other options for constructing a *TxDb*. For the *known genes* track from the UCSC Genome Browser (Kent et al. 2002), one can use the pre-built Transcript DataBase: *[TxDb.Hsapiens.UCSC.hg19.knownGene](http://bioconductor.org/packages/release/data/annotation/html/TxDb.Hsapiens.UCSC.hg19.knownGene.html)*. If the annotation file is accessible from *[AnnotationHub](http://bioconductor.org/packages/release/bioc/html/AnnotationHub.html)* (as is the case for the Ensembl genes), a pre-scanned GTF file can be imported using *makeTxDbFromGRanges*. Finally, the *makeTxDbFromBiomart* function can be used to automatically pull a gene model from Biomart using *[biomaRt](http://bioconductor.org/packages/release/bioc/html/biomaRt.html)* (Durinck et al. 2009).

Here we will demonstrate loading from a GTF file:

In [ ]:
library("GenomicFeatures")

We indicate that none of our sequences (chromosomes) are circular using a 0-length character vector.

In [ ]:
gtffile <- file.path(dir,"Homo_sapiens.GRCh37.75_subset.gtf")
(txdb <- makeTxDbFromGFF(gtffile, format="gtf", circ_seqs=character()))

The following line produces a *GRangesList* of all the exons grouped by gene.

In [ ]:
(ebg <- exonsBy(txdb, by="gene"))

### Read counting step

After these preparations, the actual counting is easy. The function *summarizeOverlaps* from the *[GenomicAlignments](http://bioconductor.org/packages/release/bioc/html/GenomicAlignments.html)* package will do this. This produces a *SummarizedExperiment* object, which contains a variety of information about the experiment, and will be described in more detail below.

**Note:** If it is desired to perform counting using multiple cores, one can use the *register* and *MulticoreParam* or *SnowParam* functions from the *[BiocParallel](http://bioconductor.org/packages/release/bioc/html/BiocParallel.html)* package before the counting call below. Expect that the `summarizeOverlaps` call will take at least 30 minutes per file for a human RNA-seq file with 20 million reads. By sending the files to separate cores, one can speed up the entire counting process.

In [ ]:
library("GenomicAlignments")
library("BiocParallel")

Here we specify to use one core, not multiple cores. We could have also skipped this line and it would also run in serial.

In [ ]:
register(SerialParam())

In [ ]:
se <- summarizeOverlaps(features=ebg, reads=bamfiles,
                        mode="Union",
                        singleEnd=FALSE,
                        ignore.strand=TRUE,
                        fragments=TRUE )

We specify a number of arguments besides the `features` and the `reads`. The `mode` argument describes what kind of read overlaps will be counted. These modes are shown in Figure 1 of the "Counting reads with summarizeOverlaps" vignette for the `r Biocpkg("GenomicAlignments")` package. Setting `singleEnd` to `FALSE` indicates that the experiment produced paired-end reads, and we want to count a pair of reads (a fragment) only once toward the count for a gene.

In order to produce correct counts, it is important to know if the RNA-Seq experiment was strand-specific or not. This experiment was not strand-specific so we set `ignore.strand` to `TRUE`. The `fragments` argument can be used when `singleEnd=FALSE` to specify if unpaired reads should be counted (yes if `fragments=TRUE`).

### SummarizedExperiment

Here we show the component parts of a *SummarizedExperiment* object, and the classes derived from it, such as the *DESeqDataSet* which is explained in the next section. This container is also discussed in the latest Bioconductor paper (Huber et al. 2015). The `assay` (pink block) contains the matrix (or matrices) of data. In our case we have created a single matrix named "counts" which contains the fragment counts for each gene and sample. The `rowRanges` (blue block) contains information about the genomic ranges (Lawrence et al. 2013), and the `colData` (green block) contains information about the samples or experiments. The highlighted line in each block represents the first row (note that the first row of `colData` lines up with the first column of the `assay`).

This example code above actually only counted a small subset of fragments from the original experiment. Nevertheless, we can still investigate the resulting *SummarizedExperiment* by looking at the counts in the `assay` slot, the phenotypic data about the samples in `colData` slot (in this case an empty *DataFrame*), and the data about the genes in the `rowRanges` slot.

In [ ]:
se
dim(se)
assayNames(se)
head(assay(se), 3)
colSums(assay(se))

The `rowRanges` have their own accessor function:

In [ ]:
rowRanges(se)

Note that the `rowRanges` slot is a *GRangesList*, which contains all the information about the exons for each gene, i.e., for each row of the count matrix. It also contains metadata about the construction of the gene model in the `metadata` slot. Here we use a helpful R function, `str`, to display the metadata compactly:

In [ ]:
str(metadata(rowRanges(se)))

The `colData` also has its own accessor function:

In [ ]:
colData(se)

The `colData` slot, so far empty, should contain all the metadata. Because we used a column of `sampleTable` to produce the `bamfiles` vector, we know the columns of `se` are in the same order as the rows of `sampleTable`. We can assign the `sampleTable` as the `colData` of the summarized experiment, by converting it into a *DataFrame* and using the assignment function:

In [ ]:
(colData(se) <- DataFrame(sampleTable))

### Branching point

At this point, we have counted the fragments which overlap the genes in the gene model we specified. This is a branching point where we could use a variety of Bioconductor packages for exploration and differential expression of the count data, including *[edgeR](http://bioconductor.org/packages/release/bioc/html/edgeR.html)*, *[limma](http://bioconductor.org/packages/release/bioc/html/limma.html)*, *[DSS](http://bioconductor.org/packages/release/bioc/html/DSS.html)*, *[EBSeq](http://bioconductor.org/packages/release/bioc/html/EBSeq.html)*, *[BaySeq](http://bioconductor.org/packages/release/bioc/html/BaySeq.html)*, and more. We will continue, using *[DESeq2](http://bioconductor.org/packages/release/bioc/html/DESeq2.html)*. The *SummarizedExperiment* object is all we need to start our analysis. In the following section we will show how to use it to create the data object used by *DESeq2*.

<a id="construct"></a>

The *DESeqDataSet*, sample information, and the design formula
--------------------------------------------------------------

Bioconductor software packages often define and use a custom class for storing data, which makes sure that all the needed data slots are consistently provided and fulfill the requirements. In addition, Bioconductor has general data classes (such as the *SummarizedExperiment*) that can be used to move data between packages. Additionally, the core Bioconductor classes provide useful functionality: for example, subsetting or reordering the rows or columns of a *SummarizedExperiment* automatically subsets or reorders the associated *rowRanges* and *colData*, which can help to prevent accidentally sample swaps that would otherwise lead to spurious results. With *SummarizedExperiment* this is all taken care of behind the scenes.

In *DESeq2*, the custom class is called *DESeqDataSet*. It is built on top of the *SummarizedExperiment* class, and it is easy to convert *SummarizedExperiment* objects into *DESeqDataSet* objects, which we show below. One of the two main differences is that the `assay` slot is instead accessed using the *counts* accessor function, and the *DESeqDataSet* class enforces that the values in this matrix are non-negative integers.

A second difference is that the *DESeqDataSet* has an associated *design formula*. The experimental design is specified at the beginning of the analysis, as it will inform many of the *DESeq2* functions how to treat the samples in the analysis (one exception is the size factor estimation, i.e., the adjustment for differing library sizes, which does not depend on the design formula). The design formula tells which columns in the sample information table (`colData`) specify the experimental design and how these factors should be used in the analysis.

The simplest design formula for differential expression would be `~ condition`, where `condition` is a column in `colData(dds)` which specifies which of two (or more groups) the samples belong to. For the airway experiment, we will specify `~ cell + dex`, which means that we want to test for the effect of dexamethasone (`dex`) controlling for the effect of different cell line (`cell`). We can see each of the columns just using the `$` directly on the *SummarizedExperiment* or *DESeqDataSet*:

In [ ]:
se$cell
se$dex

**Note:** it is prefered in R that the first level of a factor be the reference level (e.g. control, or untreated samples), so we can *relevel* the `dex` factor like so:

In [ ]:
se$dex <- relevel(se$dex, "untrt")
se$dex

For running *DESeq2* models, you can use R's formula notation to express any fixed-effects experimental design. Note that *DESeq2* uses the same formula notation as, for instance, the *lm* function of base R. If the research aim is to determine for which genes the effect of treatment is different across groups, then interaction terms can be included and tested using a design such as `~ group + treatment + group:treatment`. See the manual page for `?results` for more examples. We will show how to use an interaction term to test for condition-specific changes over a time in a [time series example](#time) below.

In the following sections, we will demonstrate the construction of the *DESeqDataSet* from two starting points:

-   from a *SummarizedExperiment* object
-   from a count matrix and a sample information table

For a full example of using the *HTSeq* Python package for read counting, please see the *[pasilla](http://bioconductor.org/packages/release/data/experiment/html/pasilla.html)* vignette. For an example of generating the *DESeqDataSet* from files produced by *htseq-count*, please see the *[DESeq2](http://bioconductor.org/packages/release/bioc/html/DESeq2.html)* vignette.

### Starting from *SummarizedExperiment*

We now use R's *data* command to load a prepared *SummarizedExperiment* that was generated from the publicly available sequencing data files associated with the Himes et al. paper, described above. The steps we used to produce this object were equivalent to those you worked through in the previous sections, except that we used all the reads and all the genes. For more details on the exact steps used to create this object, type `vignette("airway")` into your R session.

In [ ]:
data("airway")
se <- airway

Again, we want to specify that `untrt` is the reference level for the dex variable:

In [ ]:
se$dex <- relevel(se$dex, "untrt")
se$dex

We can quickly check the millions of fragments which uniquely aligned to the genes (the second argument of *round* tells how many decimal points to keep).

In [ ]:
round( colSums(assay(se)) / 1e6, 1 )

Supposing we have constructed a *SummarizedExperiment* using one of the methods described in the previous section, we now need to make sure that the object contains all the necessary information about the samples, i.e., a table with metadata on the count matrix's columns stored in the `colData` slot:

In [ ]:
colData(se)

Here we see that this object already contains an informative `colData` slot -- because we have already prepared it for you, as described in the *[airway](http://bioconductor.org/packages/release/data/experiment/html/airway.html)* vignette. However, when you work with your own data, you will have to add the pertinent sample / phenotypic information for the experiment at this stage. We highly recommend keeping this information in a comma-separated value (CSV) or tab-separated value (TSV) file, which can be exported from an Excel spreadsheet, and the assign this to the `colData` slot, making sure that the rows correspond to the columns of the *SummarizedExperiment*. We made sure of this correspondence earlier by specifying the BAM files using a column of the sample table.

Once we have our fully annotated *SummarizedExperiment* object, we can construct a *DESeqDataSet* object from it, which will then form the starting point of the analysis. We add an appropriate design for the analysis:

In [ ]:
library("DESeq2")

In [ ]:
dds <- DESeqDataSet(se, design = ~ cell + dex)

If we only wanted to perform transformations and exploratory data analysis we could use a `~ 1` for the design, but we would need to remember to substitute a real design, e.g. `~ condition`, before we run *DESeq* for differential testing or else we would only be testing the intercept.

### Starting from count matrices

In this section, we will show how to build an *DESeqDataSet* supposing we only have a count matrix and a table of sample information.

**Note:** if you have prepared a *SummarizedExperiment* you should skip this section. While the previous section would be used to contruct a *DESeqDataSet* from a *SummarizedExperiment*, here we first extract the individual object (count matrix and sample info) from the *SummarizedExperiment* in order to build it back up into a new object -- only for demonstration purposes. In practice, the count matrix would either be read in from a file or perhaps generated by an R function like *featureCounts* from the *[Rsubread](http://bioconductor.org/packages/release/bioc/html/Rsubread.html)* package (Liao, Smyth, and Shi 2014).

The information in a *SummarizedExperiment* object can be accessed with accessor functions. For example, to see the actual data, i.e., here, the fragment counts, we use the *assay* function. (The *head* function restricts the output to the first few lines.)

In [ ]:
countdata <- assay(se)
head(countdata, 3)

In this count matrix, each row represents an Ensembl gene, each column a sequenced RNA library, and the values give the raw numbers of fragments that were uniquely assigned to the respective gene in each library. We also have information on each of the samples (the columns of the count matrix). If you've counted reads with some other software, it is very importnat to to check that the columns of the count matrix correspond to the rows of the sample information table.

In [ ]:
coldata <- colData(se)

We now have all the ingredients to prepare our data object in a form that is suitable for analysis, namely:

-   `countdata`: a table with the fragment counts
-   `coldata`: a table with information about the samples

To now construct the *DESeqDataSet* object from the matrix of counts and the sample information table, we use:

In [ ]:
(ddsMat <- DESeqDataSetFromMatrix(countData = countdata,
                                  colData = coldata,
                                  design = ~ cell + dex))

We will continue with the object generated from the *SummarizedExperiment* section.

<a id="eda"></a>

Exploratory analysis and visualization
--------------------------------------

There are two separate paths in this workflow, the one we will see first involves **transformations of the counts** in order to visualizally explore sample relationships. In the second part, we will go back to the orignal raw counts for **statistical testing**. This is critical because the statistical testing methods rely on original count data (not scaled or transformed) for calculating the precision of measurements.

### Pre-filtering the dataset

Our count matrix with our *DESeqDataSet* contains many rows with only zeros, and additionally many rows with only a few fragments total. In order to reduce the size of the object, and to increase the speed of our functions, we can remove the rows which have no or nearly no information about the amount of gene expression. Here we remove rows of the *DESeqDataSet* which have no counts, or only a single count across all samples:

In [ ]:
nrow(dds)
dds <- dds[ rowSums(counts(dds)) > 1, ]
nrow(dds)

### The rlog transformation

Many common statistical methods for exploratory analysis of multidimensional data, for example clustering and *principal components analysis* (PCA), work best for data which generally has the same range of variance at different ranges of the mean values. When the expected amount of variance is approximately the same across different mean values, the data is said to be *homoskedastic*. For RNA-Seq raw counts, however, the variance grows with the mean. For example, if one performs PCA directly on a matrix of size-factor-normalized read counts, the result typically depends only on the few most strongly expressed genes because they show the largest absolute differences between samples. A simple and often used strategy to avoid this is to take the logarithm of the normalized count values plus a small pseudocount; however, now the genes with the very lowest counts will tend to dominate the results because, due to the strong Poisson noise inherent to small count values, and the fact that the logarithm amplifies differences for the smallest values, these low count genes will show the strongest relative differences between samples.

As a solution, *DESeq2* offers transformations for count data, which stabilize the variance across the mean. One such transformation is the *regularized-logarithm transformation* or *rlog* (Love, Huber, and Anders 2014). For genes with high counts, the rlog transformation will give similar result to the ordinary log2 transformation of normalized counts. For genes with lower counts, however, the values are shrunken towards the genes' averages across all samples. Using an empirical Bayesian prior on inter-sample differences in the form of a *ridge penalty*, the rlog-transformed data then becomes approximately *homoskedastic*, and can be used directly for computing distances between samples and making PCA plots. Another transformation, the *variance stabilizing transformation* (Anders and Huber 2010), is discussed alongside the *rlog* in the *DESeq2* vignette.

**Note:** the rlog transformation is provided for applications *other* than differential testing. For differential testing we recommend the *DESeq* function applied to raw counts, as described later in this workflow, which also takes into account the dependence of the variance of counts on the mean value during the dispersion estimation step.

The function *rlog* returns a *SummarizedExperiment* object which contains the rlog-transformed values in its *assay* slot.

In [ ]:
rld <- rlog(dds, blind=FALSE)
head(assay(rld), 3)

We specify `blind=FALSE`, which means that differences between cell lines and treatment should not be considered to add to the variance-mean profile of the experiment. However, the experimental design is not used directly in the transformation. For a fully *unsupervised* transformation, one can set `blind=TRUE` (which is the default).

**Note:** for large datasets (hundreds of samples), the variance stabilizing transformation will be faster to compute.

To show the effect of the transformation, we plot the first sample against the second, first simply using the *log2* function (after adding 1, to avoid taking the log of zero), and then using the rlog-transformed values. For the *log2* approach, we need to first estimate *size factors* to account for sequencing depth, and then specify `normalized=TRUE`. Sequencing depth correction is done automatically for the *rlog* method (and for *varianceStabilizingTransformation*).

In [ ]:
IRkernel::set_plot_options(width=7, height=3.75)

In [ ]:
par( mfrow = c( 1, 2 ) )
dds <- estimateSizeFactors(dds)
plot(log2(counts(dds, normalized=TRUE)[,1:2] + 1),
     pch=16, cex=0.3)
plot(assay(rld)[,1:2],
     pch=16, cex=0.3)

We can see how genes with low counts (bottom left-hand corner) seem to be excessively variable on the ordinary logarithmic scale, while the rlog transform compresses differences for the low count genes for which the data provide little information about differential expression.

### Sample distances

A useful first step in an RNA-Seq analysis is often to assess overall similarity between samples: Which samples are similar to each other, which are different? Does this fit to the expectation from the experiment's design?

We use the R function *dist* to calculate the Euclidean distance between samples. To ensure we have a roughly equal contribution from all genes, we use it on the rlog-transformed data. We need to transpose the matrix of values using *t*, because the *dist* function expects the different samples to be rows of its argument, and different dimensions (here, genes) to be columns.

In [ ]:
sampleDists <- dist( t( assay(rld) ) )
sampleDists

We visualize the distances in a heatmap, using the function *pheatmap* from the *[pheatmap](http://cran.fhcrc.org/web/packages/pheatmap/index.html)* package.

In [ ]:
library("pheatmap")
library("RColorBrewer")

In order to plot the sample distance matrix with the rows/columns arranged by the distances in our distance matrix, we manually provide `sampleDists` to the `clustering_distance` argument of the *pheatmap* function. Otherwise the *pheatmap* function would assume that the matrix contains the data values themselves, and would calculate distances between the rows/columns of the distance matrix, which is not desired. We also manually specify a blue color palette using the *colorRampPalette* function from the *[RColorBrewer](http://cran.fhcrc.org/web/packages/RColorBrewer/index.html)* package.

In [ ]:
IRkernel::set_plot_options(width=6, height=4.5)

In [ ]:
sampleDistMatrix <- as.matrix( sampleDists )
rownames(sampleDistMatrix) <- paste( rld$dex, rld$cell, sep="-" )
colnames(sampleDistMatrix) <- NULL
colors <- colorRampPalette( rev(brewer.pal(9, "Blues")) )(255)
pheatmap(sampleDistMatrix,
         clustering_distance_rows=sampleDists,
         clustering_distance_cols=sampleDists,
         col=colors)

Note that we have changed the row names of the distance matrix to contain treatment type and patient number instead of sample ID, so that we have all this information in view when looking at the heatmap.

Another option for calculating sample distances is to use the *Poisson Distance* (Witten 2011), implemented in the CRAN package *[PoiClaClu](http://cran.fhcrc.org/web/packages/PoiClaClu/index.html)*. This measure of dissimilarity between counts also takes the inherent variance structure of counts into consideration when calculating the distances between samples. The *PoissonDistance* function takes the original count matrix (not normalized) with samples as rows instead of columns, so we need to transpose the counts in `dds`.

In [ ]:
library("PoiClaClu")
poisd <- PoissonDistance(t(counts(dds)))

We can plot the heatmap as before:

In [ ]:
IRkernel::set_plot_options(width=6, height=4.5)

In [ ]:
samplePoisDistMatrix <- as.matrix( poisd$dd )
rownames(samplePoisDistMatrix) <- paste( rld$dex, rld$cell, sep="-" )
colnames(samplePoisDistMatrix) <- NULL
pheatmap(samplePoisDistMatrix,
         clustering_distance_rows=poisd$dd,
         clustering_distance_cols=poisd$dd,
         col=colors)

### PCA plot

Another way to visualize sample-to-sample distances is a principal components analysis (PCA). In this ordination method, the data points (here, the samples) are projected onto the 2D plane such that they spread out in the two directions which explain most of the differences. The x-axis is the direction which separates the data points the most. The values of the samples in this direction are written *PC1*. The y-axis is a direction (it must be *orthogonal* to the first direction) which separates the data the second most. The values of the samples in this direction are written *PC2*. The amount of the total variance which is contained in the direction is printed in the axis label. Note that these percents do not add to 100%, because there are more dimensions which contain the remaining percents (although each of these will explain less than the two we see).

In [ ]:
IRkernel::set_plot_options(width=6, height=4.5)

In [ ]:
plotPCA(rld, intgroup = c("dex", "cell"))

Here, we have used the function *plotPCA* which comes with *DESeq2*. The two terms specified by `intgroup` are the interesting groups for labeling the samples; they tell the function to use them to choose colors. We can also build the PCA plot from scratch using *[ggplot2](http://cran.fhcrc.org/web/packages/ggplot2/index.html)* (Wickham 2009). This is done by asking the *plotPCA* function to return the data used for plotting rather than building the plot. See the *ggplot2* [documentation](http://docs.ggplot2.org/current/) for more details on using *ggplot*.

In [ ]:
(data <- plotPCA(rld, intgroup = c( "dex", "cell"), returnData=TRUE))
percentVar <- round(100 * attr(data, "percentVar"))

We can then use this data to build up the plot, specifying that the color of the points should reflect dexamethasone treatment and the shape should reflect the cell line.

In [ ]:
library("ggplot2")

In [ ]:
IRkernel::set_plot_options(width=6, height=4.5)

In [ ]:
ggplot(data, aes(PC1, PC2, color=dex, shape=cell)) + geom_point(size=3) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance"))

From the PCA plot, we see that the differences between cells (the different plotting shapes) are considerable, though not stronger than the differences due to treatment with dexamethasone (red vs blue color). This shows why it will be important to account for this in differential testing by using a paired design

("paired", because each dex treated sample is paired with one untreated sample from the *same* cell line). We are already set up for this design by assigning the formula `~ cell + dex` earlier.

### MDS plot

Another plot, very similar to the PCA plot, can be made using the *multidimensional scaling* (MDS) function in base R. This is useful when we don't have a matrix of data, but only a matrix of distances. Here we compute the MDS plot for the distances calculated from the *rlog* transformed counts:

In [ ]:
IRkernel::set_plot_options(width=6, height=4.5)

In [ ]:
mds <- data.frame(cmdscale(sampleDistMatrix))
mds <- cbind(mds, as.data.frame(colData(rld)))
ggplot(mds, aes(X1,X2,color=dex,shape=cell)) + geom_point(size=3)

And here for the *PoissonDistance*:

In [ ]:
IRkernel::set_plot_options(width=6, height=4.5)

In [ ]:
mdspois <- data.frame(cmdscale(samplePoisDistMatrix))
mdspois <- cbind(mds, as.data.frame(colData(dds)))
ggplot(mdspois, aes(X1,X2,color=dex,shape=cell)) + geom_point(size=3)

<a id="de"></a>

Differential expression analysis
--------------------------------

### Running the differential expression pipeline

As we have already specified an experimental design when we created the *DESeqDataSet*, we can run the differential expression pipeline on the raw counts with a single call to the function *DESeq*:

In [ ]:
dds <- DESeq(dds)

This function will print out a message for the various steps it performs. These are described in more detail in the manual page for *DESeq*, which can be accessed by typing `?DESeq`. Briefly these are: the estimation of size factors (which control for differences in the sequencing depth of the samples), the estimation of dispersion values for each gene, and fitting a generalized linear model.

A *DESeqDataSet* is returned which contains all the fitted parameters within it, and the following section describes how to extract out results tables of interest from this object.

### Building the results table

Calling *results* without any arguments will extract the estimated log2 fold changes and *p* values for the last variable in the design formula. If there are more than 2 levels for this variable, *results* will extract the results table for a comparison of the last level over the first level. This comparison is printed at the top of the output: `dex trt vs untrt`.

In [ ]:
(res <- results(dds))

As `res` is a *DataFrame* object, it carries metadata with information on the meaning of the columns:

In [ ]:
mcols(res, use.names=TRUE)

The first column, `baseMean`, is a just the average of the normalized count values, dividing by size factors, taken over all samples in the *DESeqDataSet*. The remaining four columns refer to a specific contrast, namely the comparison of the `trt` level over the `untrt` level for the factor variable `dex`. We will find out below how to obtain other contrasts.

The column `log2FoldChange` is the effect size estimate. It tells us how much the gene's expression seems to have changed due to treatment with dexamethasone in comparison to untreated samples. This value is reported on a logarithmic scale to base 2: for example, a log2 fold change of 1.5 means that the gene's expression is increased by a multiplicative factor of \\( 2^{1.5} 2.82 \\).

Of course, this estimate has an uncertainty associated with it, which is available in the column `lfcSE`, the standard error estimate for the log2 fold change estimate. We can also express the uncertainty of a particular effect size estimate as the result of a statistical test. The purpose of a test for differential expression is to test whether the data provides sufficient evidence to conclude that this value is really different from zero. *DESeq2* performs for each gene a *hypothesis test* to see whether evidence is sufficient to decide against the *null hypothesis* that there is zero effect of the treatment on the gene and that the observed difference between treatment and control was merely caused by experimental variability (i.e., the type of variability that you can expect between different samples in the same treatment group). As usual in statistics, the result of this test is reported as a *p* value, and it is found in the column `pvalue`. Remember that a *p* value indicates the probability that a fold change as strong as the observed one, or even stronger, would be seen under the situation described by the null hypothesis.

We can also summarize the results with the following line of code, which reports some additional information, which will be covered in later sections.

In [ ]:
summary(res)

Note that there are many genes with differential expression due to dexamethasone treatment at the FDR level of 10%. This makes sense, as the smooth muscle cells of the airway are known to react to glucocorticoid steroids. However, there are two ways to be more strict about which set of genes are considered significant:

-   lower the false discovery rate threshold (the threshold on `padj` in the results table)
-   raise the log2 fold change threshold from 0 using the `lfcThreshold` argument of *results*

If we lower the false discovery rate threshold, we should also tell this value to `results()`, so that the function will use an alternative threshold for the optimal independent filtering step:

In [ ]:
res.05 <- results(dds, alpha=.05)
table(res.05$padj < .05)

If we want to raise the log2 fold change threshold, so that we test for genes which show more substantial changes due to treatment, we simply supply a value on the log2 scale. For example, by specifying `lfcThreshold=1`, we test for genes which show significant effects of treatment which are more than doubling or less than halving, because \\( 2^1 = 2 \\).

In [ ]:
resLFC1 <- results(dds, lfcThreshold=1)
table(resLFC1$padj < 0.1)

Sometimes a subset of the *p* values in `res` will be `NA` ("not available"). This is *DESeq*'s way of reporting that all counts for this gene were zero, and hence not test was applied. In addition, *p* values can be assigned `NA` if the gene was excluded from analysis because it contained an extreme count outlier. For more information, see the outlier detection section of the *DESeq2* vignette.

If you use the results from an R analysis package in published research, you can find the proper citation for the software by typing `citation("pkgName")`, where you would substitute the name of the package for `pkgName`. Citing methods papers helps to support and reward the individuals who put time into open source software for genomic data analysis.

### Other comparisons

In general, the results for a comparison of any two levels of a variable can be extracted using the `contrast` argument to *results*. The user should specify three values: the name of the variable, the name of the level for the numerator, and the name of the level for the denominator. Here we extract results for the log2 of the fold change of one cell line over another:

In [ ]:
results(dds, contrast=c("cell", "N061011", "N61311"))

If results for an interaction term are desired, the `name` argument of *results* should be used. Please see the help for the *results* function for more details.

### Multiple testing

In high-throughput biology, we are careful to not use the *p* values directly as evidence against the null, but to correct for *multiple testing*. What would happen if we were to simply threshold the *p* values at a low value, say 0.05? There are 5722 genes with a *p* value below 0.05 among the 29391 genes, for which the test succeeded in reporting a *p* value:

In [ ]:
sum(res$pvalue < 0.05, na.rm=TRUE)
sum(!is.na(res$pvalue))

Now, assume for a moment that the null hypothesis is true for all genes, i.e., no gene is affected by the treatment with dexamethasone. Then, by the definition of *p* value, we expect up to 5% of the genes to have a *p* value below 0.05. This amounts to 1470 genes. If we just considered the list of genes with a *p* value below 0.05 as differentially expressed, this list should therefore be expected to contain up to 1470 / 5722 = 26% false positives.

*DESeq2* uses the Benjamini-Hochberg (BH) adjustment (Benjamini and Hochberg 1995) as described in the base R *p.adjust* function; in brief, this method calculates for each gene an adjusted *p* value which answers the following question: if one called significant all genes with an adjusted *p* value less than or equal to this gene's adjusted *p* value threshold, what would be the fraction of false positives (the *false discovery rate*, FDR) among them, in the sense of the calculation outlined above? These values, called the BH-adjusted *p* values, are given in the column `padj` of the `res` object.

The FDR is a useful statistic for many high-throughput experiments, as we are often interested in reporting or focusing on a set of interesting genes, and we would like to put an upper bound the percent of false positives in this set.

Hence, if we consider a fraction of 10% false positives acceptable, we can consider all genes with an adjusted *p* value below 10% = 0.1 as significant. How many such genes are there?

In [ ]:
sum(res$padj < 0.1, na.rm=TRUE)

We subset the results table to these genes and then sort it by the log2 fold change estimate to get the significant genes with the strongest down-regulation:

In [ ]:
resSig <- subset(res, padj < 0.1)
head(resSig[ order(resSig$log2FoldChange), ])

...and with the strongest up-regulation:

In [ ]:
head(resSig[ order(resSig$log2FoldChange, decreasing=TRUE), ])

<a id="plots"></a>

Plotting results
----------------

A quick way to visualize the counts for a particular gene is to use the *plotCounts* function, which takes as arguments the *DESeqDataSet*, a gene name, and the group over which to plot the counts.

In [ ]:
topGene <- rownames(res)[which.min(res$padj)]
plotCounts(dds, gene=topGene, intgroup=c("dex"))

We can also make custom plots using the *ggplot* function from the *[ggplot2](http://cran.fhcrc.org/web/packages/ggplot2/index.html)* package:

In [ ]:
IRkernel::set_plot_options(width=6, height=4.5)

In [ ]:
data <- plotCounts(dds, gene=topGene, intgroup=c("dex","cell"), returnData=TRUE)
ggplot(data, aes(x=dex, y=count, color=cell)) +
  scale_y_log10() + 
  geom_point(position=position_jitter(width=.1,height=0), size=3)

Here we use a more structural arrangement instead of random jitter, and color by the treatment:

In [ ]:
IRkernel::set_plot_options(width=6, height=4.5)

In [ ]:
ggplot(data, aes(x=dex, y=count, fill=dex)) +
  scale_y_log10() + 
  geom_dotplot(binaxis="y", stackdir="center")

Note that the *DESeq* test actually takes into account the cell line effect, so a more accurate plot would also show lines between samples from the same cell line:

In [ ]:
IRkernel::set_plot_options(width=6, height=4.5)

In [ ]:
ggplot(data, aes(x=dex, y=count, color=cell, group=cell)) +
  scale_y_log10() + geom_point(size=3) + geom_line()

An *MA-plot* (R. Dudoit et al. 2002) provides a useful overview for an experiment with a two-group comparison. The log2 fold change for a particular comparison is plotted on the y-axis and the average of the counts normalized by size factor is shown on the x-axis ("M" for minus, because a log ratio is equal to log minus log, and "A" for average).

In [ ]:
plotMA(res, ylim=c(-5,5))

Each gene is represented with a dot. Genes with an adjusted *p* value below a threshold (here 0.1, the default) are shown in red. The *DESeq2* package incorporates a prior on log2 fold changes, resulting in moderated log2 fold changes from genes with low counts and highly variable counts, as can be seen by the narrowing of spread of points on the left side of the plot. This plot demonstrates that only genes with a large average normalized count contain sufficient information to yield a significant call.

We can also make the MA-plot for the results table in which we raised the log2 fold change threshold:

In [ ]:
plotMA(resLFC1, ylim=c(-5,5))

We can label individual points on the MA plot as well. Here we use the *with* R function to plot a circle and text for a selected row of the results object. Within the *with* function, only the `baseMean` and `log2FoldChange` values for the selected rows of `res` are used.

In [ ]:
plotMA(resLFC1, ylim=c(-5,5))
topGene <- rownames(resLFC1)[which.min(resLFC1$padj)]
with(resLFC1[topGene, ], {
  points(baseMean, log2FoldChange, col="dodgerblue", cex=2, lwd=2)
  text(baseMean, log2FoldChange, topGene, pos=2, col="dodgerblue")
})

Another useful diagnostic plot is the histogram of the *p* values.

In [ ]:
hist(res$pvalue, breaks=0:20/20, col="grey50", border="white")

This plot becomes smoother by excluding genes with very small counts, which generate the spikes in the original plot:

In [ ]:
hist(res$pvalue[res$baseMean > 1], breaks=0:20/20, col="grey50", border="white")

Gene clustering
---------------

In the sample distance heatmap made previously, the dendrogram at the side shows us a hierarchical clustering of the samples. Such a clustering can also be performed for the genes. Since the clustering is only relevant for genes that actually carry signal, one usually would only cluster a subset of most highly variable genes. Here, for demonstration, let us select the 20 genes with the highest variance across samples. We will work with the *rlog* transformed counts:

In [ ]:
library("genefilter")
topVarGenes <- head(order(rowVars(assay(rld)),decreasing=TRUE),20)

The heatmap becomes more interesting if we do not look at absolute expression strength but rather at the amount by which each gene deviates in a specific sample from the gene's average across all samples. Hence, we center each genes' values across samples, and plot a heatmap. We provide a *data.frame* which instructs the *pheatmap* function how to label the columns.

In [ ]:
mat <- assay(rld)[ topVarGenes, ]
mat <- mat - rowMeans(mat)
df <- as.data.frame(colData(rld)[,c("cell","dex")])
pheatmap(mat, annotation_col=df)

We can now see blocks of genes which covary across patients. Note that a set of genes at the top of the heatmap are separating the N061011 cell line from the others. In the center of the heatmap, we see a set of genes for which the dexamethasone treated samples have higher gene expression.

Independent filtering
---------------------

The MA plot highlights an important property of RNA-Seq data. For weakly expressed genes, we have no chance of seeing differential expression, because the low read counts suffer from such high Poisson noise that any biological effect is drowned in the uncertainties from the sampling at a low rate. We can also show this by examining the ratio of small *p* values (say, less than, 0.05) for genes binned by mean normalized count. We will use the thresholded results table to show what this looks like in a case when there are few tests with small *p* value.

In [ ]:
# create bins using the quantile function
qs <- c(0, quantile(resLFC1$baseMean[resLFC1$baseMean > 0], 0:7/7))
# cut the genes into the bins
bins <- cut(resLFC1$baseMean, qs)
# rename the levels of the bins using the middle point
levels(bins) <- paste0("~",round(.5*qs[-1] + .5*qs[-length(qs)]))
# calculate the ratio of $p$ values less than .05 for each bin
ratios <- tapply(resLFC1$pvalue, bins, function(p) mean(p < .05, na.rm=TRUE))
# plot these ratios
barplot(ratios, xlab="mean normalized count", ylab="ratio of small p values")

At first sight, there may seem to be little benefit in filtering out these genes. After all, the test found them to be non-significant anyway. However, these genes have an influence on the multiple testing adjustment, whose performance improves if such genes are removed. By removing the low count genes from the input to the FDR procedure, we can find more genes to be significant among those which we keep, and so improved the power of our test. This approach is known as *independent filtering*.

The *DESeq2* software automatically performs independent filtering which maximizes the number of genes which will have adjusted *p* value less than a critical value (by default, `alpha` is set to 0.1). This automatic independent filtering is performed by, and can be controlled by, the *results* function. We can observe how the number of rejections changes for various cutoffs based on quantiles of the mean normalized count. The following optimal threshold and table of possible values is stored as an attribute of the results object.

In [ ]:
attr(resLFC1,"filterThreshold")
plot(attr(resLFC1,"filterNumRej"), type="b",
     xlab="quantiles of 'baseMean'",
     ylab="number of rejections")

The term *independent* highlights an important caveat. Such filtering is permissible only if the statistic which we filter on (here the mean of normalized counts across all samples) is independent of the actual test statistic (the *p* value) under the null hypothesis. Otherwise, the filtering would invalidate the test and consequently the assumptions of the BH procedure. The independent filtering software used inside *DESeq2* comes from the *[genefilter](http://bioconductor.org/packages/release/bioc/html/genefilter.html)* package, which contains a reference to a paper describing the statistical foundation for independent filtering (Bourgon, Gentleman, and Huber 2010).

<a id="annotate"></a>

Annotation: adding gene names
-----------------------------

Our result table so far only contains information about Ensembl gene IDs, but alternative gene names may be more informative for collaborators. Bioconductor's annotation packages help with mapping various ID schemes to each other. We load the *[AnnotationDbi](http://bioconductor.org/packages/release/bioc/html/AnnotationDbi.html)* package and the annotation package *[org.Hs.eg.db](http://bioconductor.org/packages/release/data/annotation/html/org.Hs.eg.db.html)*:

In [ ]:
library("AnnotationDbi")
library("org.Hs.eg.db")

This is the organism annotation package ("org") for *Homo sapiens* ("Hs"), organized as an *AnnotationDbi* database package ("db"), using Entrez Gene IDs ("eg") as primary key. To get a list of all available key types, use:

In [ ]:
columns(org.Hs.eg.db)

We can use the *mapIds* function to add invidual columns to our results table. We provide the row names of our results table as a key, and specify that `keytype=ENSEMBL`. The `column` argument tells the *mapIds* function which information we want, and the `multiVals` argument tells the function what to do if there are multiple possible values for a single input value. Here we ask to just give us back the first one which occurs in the database. To add the gene symbol and Entrez ID, we call *mapIds* twice.

In [ ]:
res$symbol <- mapIds(org.Hs.eg.db,
                     keys=row.names(res),
                     column="SYMBOL",
                     keytype="ENSEMBL",
                     multiVals="first")
res$entrez <- mapIds(org.Hs.eg.db,
                     keys=row.names(res),
                     column="ENTREZID",
                     keytype="ENSEMBL",
                     multiVals="first")

Now the results have the desired external gene IDs:

In [ ]:
resOrdered <- res[order(res$padj),]
head(resOrdered)

Exporting results
-----------------

You can easily save the results table in a CSV file, which you can then share or load with a spreadsheet program such as Excel. The call to *as.data.frame* is necessary to convert the *DataFrame* object (*[IRanges](http://bioconductor.org/packages/release/bioc/html/IRanges.html)* package) to a *data.frame* object which can be processed by *write.csv*.

In [ ]:
resOrderedDF <- as.data.frame(resOrdered)[1:100,]
write.csv(resOrderedDF, file="results.csv")

Another more sophisticated package for exporting results from various Bioconductor analysis packages is the *[ReportingTools](http://bioconductor.org/packages/release/bioc/html/ReportingTools.html)* package (Huntley et al. 2013). *ReportingTools* will automatically generate dynamic HTML documents, including links to external databases using gene identifiers and boxplots summarizing the normalized counts across groups. See the *ReportingTools* vignettes for full details. The simplest version of creating a dynamic *ReportingTools* report is performed with the following code:

In [ ]:
library("ReportingTools")
htmlRep <- HTMLReport(shortName="report", title="My report",
                      reportDirectory="./report")
publish(resOrderedDF, htmlRep)
url <- finish(htmlRep)
browseURL(url)

Plotting fold changes in genomic space
--------------------------------------

If we have used the *summarizeOverlaps* function to count the reads, then our *DESeqDataSet* object is built on top of ready-to-use Bioconductor objects specifying the genomic ranges of the genes. We can therefore easily plot our differential expression results in genomic space. While the *results* function by default returns a *DataFrame*, using the `format` argument, we can ask for *GRanges* or *GRangesList* output.

In [ ]:
(resGR <- results(dds, lfcThreshold=1, format="GRanges"))

We need to add the symbol again for labeling the genes on the plot:

In [ ]:
resGR$symbol <- mapIds(org.Hs.eg.db, names(resGR), "SYMBOL", "ENSEMBL")

We will use the *[Gviz](http://bioconductor.org/packages/release/bioc/html/Gviz.html)* package for plotting the GRanges and associated metadata: the log fold changes due to dexamethasone treatment.

In [ ]:
library("Gviz")

The following code chunk specifies a window of 1 million base pairs upstream and downstream from the gene with the smallest *p* value. We create a subset of our full results, for genes within the window We add the gene symbol as a name, if the symbol exists or is not duplicated in our subset.

In [ ]:
window <- resGR[topGene] + 1e6
strand(window) <- "*"
resGRsub <- resGR[resGR %over% window]
naOrDup <- is.na(resGRsub$symbol) | duplicated(resGRsub$symbol)
resGRsub$group <- ifelse(naOrDup, names(resGRsub), resGRsub$symbol)

We create a vector specifying if the genes in this subset had a low false discovery rate.

In [ ]:
sig <- factor(ifelse(resGRsub$padj < .1 & !is.na(resGRsub$padj),"sig","notsig"))

We can then plot the results using *[Gviz](http://bioconductor.org/packages/release/bioc/html/Gviz.html)* functions. We create an axis track specifying our location in the genome, a track which will show the genes and their names, colored by significance, and a data track which will draw vertical bars showing the moderated log fold change produced by *DESeq2*, which we know are only large when the effect is well supported by the information in the counts.

In [ ]:
options(ucscChromosomeNames=FALSE)
g <- GenomeAxisTrack()
a <- AnnotationTrack(resGRsub, name="gene ranges", feature=sig)
d <- DataTrack(resGRsub, data="log2FoldChange", baseline=0,
               type="h", name="log2 fold change", strand="+")
plotTracks(list(g,d,a), groupAnnotation="group", notsig="grey", sig="hotpink")

<a id="batch"></a>

Removing hidden batch effects
-----------------------------

Suppose we did not know that there were different cell lines involved in the experiment, only that there was treatment with dexamethasone. The cell line effect on the counts then would represent some hidden and unwanted variation which might be affecting many or all of the genes in the dataset. We can use statistical methods designed for RNA-seq from the *[sva](http://bioconductor.org/packages/release/bioc/html/sva.html)* package (Leek 2014) to detect such groupings of the samples, and then we can add these to the *DESeqDataSet* design, in order to account for them. The *SVA* package uses the term *surrogate variables* for the estimated variables which we want to account for in our analysis. Another package for detecting hidden batches is the *[RUVSeq](http://bioconductor.org/packages/release/bioc/html/RUVSeq.html)* package (Risso et al. 2014), with the acronym "Remove Unwanted Variation".

In [ ]:
library("sva")

Below we obtain a matrix of normalized counts for which the average count across samples is larger than 1. As we described above, we are trying to recover any hidden batch effects, supposing that we do not know the cell line information. So we use a full model matrix with the *dex* variable, and a reduced, or null, model matrix with only an intercept term. Finally we specify that we want to estimate 2 surrogate variables. For more information read the manual page for the *svaseq* function by typing `?svaseq`.

In [ ]:
dat <- counts(dds, normalized=TRUE)
idx <- rowMeans(dat) > 1
dat <- dat[idx,]
mod <- model.matrix(~ dex, colData(dds))
mod0 <- model.matrix(~ 1, colData(dds))
svseq <- svaseq(dat, mod, mod0, n.sv=2)
svseq$sv

Because we actually do know the cell lines, we can see how well the SVA method did at recovering these variables:

In [ ]:
par(mfrow=c(2,1),mar=c(3,5,3,1))
stripchart(svseq$sv[,1] ~ dds$cell,vertical=TRUE,main="SV1")
abline(h=0)
stripchart(svseq$sv[,2] ~ dds$cell,vertical=TRUE,main="SV2")
abline(h=0)

Finally, in order to use SVA to remove any effect on the counts from our surrogate variables, we simply add these two surrogate variables as columns to the *DESeqDataSet* and then add them to the design:

In [ ]:
ddssva <- dds
ddssva$SV1 <- svseq$sv[,1]
ddssva$SV2 <- svseq$sv[,2]
design(ddssva) <- ~ SV1 + SV2 + dex

We could then produce results controlling for surrogate variables by running *DESeq* with the new design:

In [ ]:
ddssva <- DESeq(ddssva)

<a id="time"></a>

Time series experiments
-----------------------

*DESeq2* can be used to analyze time series experiments, for example to find those genes which react in a condition-specific manner over time, compared to a set of baseline samples. Here we demonstrate a basic time series analysis with the *[fission](http://bioconductor.org/packages/release/data/experiment/html/fission.html)* data package, which contains gene counts for an RNA-Seq time course of fission yeast (Leong et al. 2014). The yeast were exposed to oxidative stress, and half of the samples contain a deletion of the gene *atf21*. We use a design which models the strain difference at time 0, the difference over time, and any strain-specific differences over time (the interaction term `strain:minute`).

In [ ]:
library("fission")
data("fission")
ddsTC <- DESeqDataSet(fission, ~ strain + minute + strain:minute)

The following chunk of code performs a likelihood ratio test, where we remove the strain-specific differences over time. Genes with small *p* values from this test are those which, at one or more time points after time 0 showed a strain-specific effect. Note therefore that this will not give small *p* values to genes which moved up or down over time in the same way in both strains.

In [ ]:
ddsTC <- DESeq(ddsTC, test="LRT", reduced = ~ strain + minute)
resTC <- results(ddsTC)
resTC$symbol <- mcols(ddsTC)$symbol
head(resTC[order(resTC$pvalue),],4)

This is just one of the tests which can be applied to time series data. Another option would be to model the counts as a smooth function of time, and to include an interaction term of the condition with the smooth function. It is possible to build such a model using spline basis functions within R.

We can plot the counts for the groups over time using *[ggplot2](http://cran.fhcrc.org/web/packages/ggplot2/index.html)*, for the gene with the smallest *p* value, testing for condition-dependent time profile and accounting for differences at time 0. Keep in mind that the interaction terms are the *difference* between the two groups at a given time after accounting for the difference at time 0.

In [ ]:
IRkernel::set_plot_options(width=6, height=4.5)

In [ ]:
data <- plotCounts(ddsTC, which.min(resTC$pvalue), 
                   intgroup=c("minute","strain"), returnData=TRUE)
ggplot(data, aes(x=minute, y=count, color=strain, group=strain)) + 
  geom_point() + stat_smooth(se=FALSE,method="loess") +  scale_y_log10()

Wald tests for the log2 fold changes at individual time points can be investigated using the `test` argument to *results*:

In [ ]:
resultsNames(ddsTC)
res30 <- results(ddsTC, name="strainmut.minute30", test="Wald")
res30[which.min(resTC$pvalue),]

We can further more cluster significant genes by their profiles. We extract a matrix of the shrunken log2 fold changes using the *coef* function:

In [ ]:
betas <- coef(ddsTC)
colnames(betas)

We can now plot the log2 fold changes in a heatmap. Notice the bottom set of genes, which show strong induction of expression for the baseline samples in minutes 15-60 (red boxes in the bottom left corner), but then have slightly differences for the mutant strain (shown in the boxes in the bottom right corner).

In [ ]:
library("pheatmap")
topGenes <- head(order(resTC$pvalue),20)
mat <- betas[topGenes, -c(1,2)]
thr <- 3 # threshold for plotting
mat[mat < -thr] <- -thr
mat[mat > thr] <- thr
pheatmap(mat, breaks=seq(from=-thr, to=thr, length=101),
         cluster_col=FALSE)

References
----------

Anders, Simon, and Wolfgang Huber. 2010. “Differential expression analysis for sequence count data.” *Genome Biology* 11 (10). BioMed Central Ltd: R106+. [doi:10.1186/gb-2010-11-10-r106](http://doi.org/10.1186/gb-2010-11-10-r106).

Anders, Simon, Paul T. Pyl, and Wolfgang Huber. 2014. “HTSeq – A Python framework to work with high-throughput sequencing data.” *BioRxiv*. http://dx.doi.org/10.1101/002824; bioRxiv. [doi:10.1101/002824](http://doi.org/10.1101/002824).

Benjamini, Yoav, and Yosef Hochberg. 1995. “Controlling the False Discovery Rate: A Practical and Powerful Approach to Multiple Testing.” *Journal of the Royal Statistical Society. Series B (Methodological)* 57 (1). Blackwell Publishing for the Royal Statistical Society: 289–300. [doi:10.2307/2346101](http://doi.org/10.2307/2346101).

Bourgon, R., R. Gentleman, and W. Huber. 2010. “Independent filtering increases detection power for high-throughput experiments.” *Proceedings of the National Academy of Sciences* 107 (21). National Academy of Sciences: 9546–51. [doi:10.1073/pnas.0914005107](http://doi.org/10.1073/pnas.0914005107).

Dobin, Alexander, Carrie A. Davis, Felix Schlesinger, Jorg Drenkow, Chris Zaleski, Sonali Jha, Philippe Batut, Mark Chaisson, and Thomas R. Gingeras. 2013. “STAR: ultrafast universal RNA-seq aligner.” *Bioinformatics* 29 (1). Oxford University Press: 15–21. [doi:10.1093/bioinformatics/bts635](http://doi.org/10.1093/bioinformatics/bts635).

Dudoit, Rine, Yee H. Yang, Matthew J. Callow, and Terence P. Speed. 2002. “Statistical methods for identifying differentially expressed genes in replicated cDNA microarray experiments.” In *Statistica Sinica*, 111–39. <http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.117.9702>.

Durinck, Steffen, Paul T. Spellman, Ewan Birney, and Wolfgang Huber. 2009. “Mapping identifiers for the integration of genomic datasets with the R/Bioconductor package biomaRt.” *Nature Protocols* 4 (8). Nature Publishing Group: 1184–91. [doi:10.1038/nprot.2009.97](http://doi.org/10.1038/nprot.2009.97).

Flicek, Paul, M. Ridwan Amode, Daniel Barrell, Kathryn Beal, Konstantinos Billis, Simon Brent, Denise Carvalho-Silva, et al. 2014. “Ensembl 2014.” *Nucleic Acids Research* 42 (D1). Oxford University Press: D749–55. [doi:10.1093/nar/gkt1196](http://doi.org/10.1093/nar/gkt1196).

Hardcastle, Thomas, and Krystyna Kelly. 2010. “baySeq: Empirical Bayesian methods for identifying differential expression in sequence count data.” *BMC Bioinformatics* 11 (1): 422+. [doi:10.1186/1471-2105-11-422](http://doi.org/10.1186/1471-2105-11-422).

Himes, Blanca E., Xiaofeng Jiang, Peter Wagner, Ruoxi Hu, Qiyu Wang, Barbara Klanderman, Reid M. Whitaker, et al. 2014. “RNA-Seq transcriptome profiling identifies CRISPLD2 as a glucocorticoid responsive gene that modulates cytokine function in airway smooth muscle cells.” *PloS One* 9 (6). [doi:10.1371/journal.pone.0099625](http://doi.org/10.1371/journal.pone.0099625).

Huber, Wolfgang, Vincent J. Carey, Robert Gentleman, Simon Anders, Marc Carlson, Benilton S. Carvalho, Hector Corrada C. Bravo, et al. 2015. “Orchestrating high-throughput genomic analysis with Bioconductor.” *Nature Methods* 12 (2). Nature Publishing Group: 115–21. [doi:10.1038/nmeth.3252](http://doi.org/10.1038/nmeth.3252).

Huntley, Melanie A., Jessica L. Larson, Christina Chaivorapol, Gabriel Becker, Michael Lawrence, Jason A. Hackney, and Joshua S. Kaminker. 2013. “ReportingTools: an automated result processing and presentation toolkit for high-throughput genomic analyses.” *Bioinformatics* 29 (24). Oxford University Press: 3220–21. [doi:10.1093/bioinformatics/btt551](http://doi.org/10.1093/bioinformatics/btt551).

Kent, W. James, Charles W. Sugnet, Terrence S. Furey, Krishna M. Roskin, Tom H. Pringle, Alan M. Zahler, and David Haussler. 2002. “The human genome browser at UCSC.” *Genome Research* 12 (6). Cold Spring Harbor Laboratory Press: 996–1006. [doi:10.1101/gr.229102](http://doi.org/10.1101/gr.229102).

Law, Charity W., Yunshun Chen, Wei Shi, and Gordon K. Smyth. 2014. “Voom: precision weights unlock linear model analysis tools for RNA-seq read counts.” *Genome Biology* 15 (2). BioMed Central Ltd: R29+. [doi:10.1186/gb-2014-15-2-r29](http://doi.org/10.1186/gb-2014-15-2-r29).

Lawrence, Michael, Wolfgang Huber, Hervé Pagès, Patrick Aboyoun, Marc Carlson, Robert Gentleman, Martin T. Morgan, and Vincent J. Carey. 2013. “Software for Computing and Annotating Genomic Ranges.” Edited by Andreas Prlic. *PLoS Computational Biology* 9 (8). Public Library of Science: e1003118+. [doi:10.1371/journal.pcbi.1003118](http://doi.org/10.1371/journal.pcbi.1003118).

Leek, Jeffrey T. 2014. “svaseq: removing batch effects and other unwanted noise from sequencing data.” *Nucleic Acids Research* 42 (21). Oxford University Press: 000. [doi:10.1093/nar/gku864](http://doi.org/10.1093/nar/gku864).

Leng, N., J. A. Dawson, J. A. Thomson, V. Ruotti, A. I. Rissman, B. M. G. Smits, J. D. Haag, M. N. Gould, R. M. Stewart, and C. Kendziorski. 2013. “EBSeq: an empirical Bayes hierarchical model for inference in RNA-seq experiments.” *Bioinformatics* 29 (8). Oxford University Press: 1035–43. [doi:10.1093/bioinformatics/btt087](http://doi.org/10.1093/bioinformatics/btt087).

Leong, Hui S., Keren Dawson, Chris Wirth, Yaoyong Li, Yvonne Connolly, Duncan L. Smith, Caroline R. Wilkinson, and Crispin J. Miller. 2014. “A global non-coding RNA system modulates fission yeast protein levels in response to stress.” *Nature Communications* 5. [doi:10.1038/ncomms4947](http://doi.org/10.1038/ncomms4947).

Li, Heng, Bob Handsaker, Alec Wysoker, Tim Fennell, Jue Ruan, Nils Homer, Gabor Marth, Goncalo Abecasis, Richard Durbin, and 1000 Genome Project Data Processing Subgroup. 2009. “The Sequence Alignment/Map format and SAMtools.” *Bioinformatics (Oxford, England)* 25 (16). Oxford University Press: 2078–79. [doi:10.1093/bioinformatics/btp352](http://doi.org/10.1093/bioinformatics/btp352).

Liao, Y., G. K. Smyth, and W. Shi. 2014. “featureCounts: an efficient general purpose program for assigning sequence reads to genomic features.” *Bioinformatics* 30 (7). Oxford University Press: 923–30. [doi:10.1093/bioinformatics/btt656](http://doi.org/10.1093/bioinformatics/btt656).

Love, Michael I., Wolfgang Huber, and Simon Anders. 2014. “Moderated estimation of fold change and dispersion for RNA-seq data with DESeq2.” *Genome Biology* 15 (12). BioMed Central Ltd: 550+. [doi:10.1186/s13059-014-0550-8](http://doi.org/10.1186/s13059-014-0550-8).

Risso, Davide, John Ngai, Terence P. Speed, and Sandrine Dudoit. 2014. “Normalization of RNA-seq data using factor analysis of control genes or samples.” *Nature Biotechnology* 32 (9). Nature Publishing Group: 896–902. [doi:10.1038/nbt.2931](http://doi.org/10.1038/nbt.2931).

Robinson, M. D., D. J. McCarthy, and G. K. Smyth. 2009. “edgeR: a Bioconductor package for differential expression analysis of digital gene expression data.” *Bioinformatics* 26 (1). Oxford University Press: 139–40. [doi:10.1093/bioinformatics/btp616](http://doi.org/10.1093/bioinformatics/btp616).

Wickham, Hadley. 2009. *ggplot2*. New York, NY: Springer New York. [doi:10.1007/978-0-387-98141-3](http://doi.org/10.1007/978-0-387-98141-3).

Witten, Daniela M. 2011. “Classification and clustering of sequencing data using a Poisson model.” *The Annals of Applied Statistics* 5 (4): 2493–2518. [doi:10.1214/11-AOAS493](http://doi.org/10.1214/11-AOAS493).

Wu, Hao, Chi Wang, and Zhijin Wu. 2013. “A new shrinkage estimator for dispersion improves differential expression detection in RNA-seq data.” *Biostatistics* 14 (2). Oxford University Press: 232–43. [doi:10.1093/biostatistics/kxs033](http://doi.org/10.1093/biostatistics/kxs033).